We perform matching order by order in a taylor series or perturbation expansion.

It seems like this could be modelled usign a mix of power serious lazy/ generator techniques and linear relations


My way of building relations is actually an equalizer?

And then also kind of 

The span thing.

Take generators, there is a map fr
This is a map from Ngen -> A
and NGen -> B

To get new NGen

A <x- Ngen -y> B

and then 
B <z- Ngen' -w> C

Then we need to find best/universal maps (matrices) such that
yf = zg  ===   [y  -z][f ; g] = 0( indeed this is an hrep equation. An equalizer? Ngen'' ==>   )

Then the new generator matrices are
xf and wg


Basically the same shit though


span is Vrep
cospan is Hrep


span composition vs cospan composition

    Escardo - 

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.48.7545&rep=rep1&type=pdf



We choose to only cares about linear transofrmations projectively. A and l A are the same transformation for our purposes. 
We are mapping subspaces to each other, not individual vectors to individual vectors.


Affine transformations may be more familiar. y = Ax + b. We can embed affine transformations into linear transformations by the method of homogenous coordinates. Homogenous coordinates

We can also 



Affine transformations and gaussians.
Multidimensional Gaussian integrals are basically solvable by linear algebra.
e^{ xAx + bx + c }

Multiply and integrate -> add and mininimize over interior variable.
-> immedaitely leads to Gaussian + linear constraints (which is fine. these are tractable)
-> Affine equations with dual variables.


We can still do summation of objective with dual variables, but it is more confusing.

Why are there so many moving parts?
It feels very inelegant.

It isn't a catgoery without linear constraints

can sympy integrate the gaussian for us?
Or Minimize for us?


Really we want guassians and delta functions
There is a classic point that really delta functions are distributions, not functions
They are linear functionals on functions

So if we defined ourselves as disribtuions, Then inplcitly we're getitng a nice beefy gaussian coming in.

There is a little more going on. It's both a central point and an ellipse around that point
 (perhaps degenrate in some dimensions)
 
 Homegenizing this becuases cones in projective space. But like good cones.


You want to come up with a functional relationship
such that the function combined with the linear relations is sufficient to determined

maybe you want to maintain a span.

a minimal representation of the space with functions going to the interface variables.

So then it is a regular span,
But we have to combine the quadratic forms of the free variables to get the combined quadratic form.


The span cospan thing makes sense because we really do need to effect linear substitions during our maniupations.


so.

vrep and hrep are not symmetrical in this case because we need to add objectives, which are quadratic forms of the vrep, but not the hrep.

two forms
v1Av1    v2Bv2

we form pullback of mapping to interior variables y
f1 : v1 -> y
f2 : v2 -> y

p1 : v -> v1
p2 : v -> v2


A' = p1Ap1 + p2BP2
g' = g1 . p1
h' = h2 . p2

If minimzatin of the inteiorr variable doesn't matter, by diconnect or imcomplete



Gaussians and 

Computing Gaussians in Numpy with Categorical Spans


Why gaussians over other options?


https://github.com/epatters/Catlab.jl/issues/87#issuecomment-596166224



In [27]:
import numpy as np
import scipy
import scipy.linalg
# https://docs.scipy.org/doc/numpy/user/basics.subclassing.html
class FinVect(np.ndarray):
    def __new__(cls, input_array, info=None):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        
        obj = np.asarray(input_array).view(cls)
        assert(len(obj.shape) == 2) # should be matrix
        # add the new attribute to the created instance
        # Finally, we must return the newly created object:
        return obj
    @property
    def dom(self):
        ''' returns the domain of the morphism. This is the number of columns of the matrix'''
        return self.shape[1]
    @property
    def cod(self):
        ''' returns the codomain of the morphism. This is the numer of rows of the matrix'''
        return self.shape[0]
    def idd(n):
        ''' The identity morphism is the identity matrix. Isn't that nice? '''
        return FinVect(np.eye(n))
    def compose(f,g):
        ''' Morphism composition is matrix multiplication. Isn't that nice?'''
        return f @ g
    def par(f,g):
        ''' One choice of monoidal product is the direct sum '''
        r, c = f.shape
        rg, cg = g.shape
        return Vect(np.block( [ [f       ,           np.zeros((r,cg))]  ,
                                [np.zeros((rg,c))  , g              ]]  ))
    def par2(f,g):
        '''  another choice is the kroncker product'''
        return np.kron(f,g)
    def monic(self):
        ''' Is mapping injective. 
            In other words, maps every incoming vector to distinct outgoing vector.
            In other words, are the columns independent. 
            In other words, does `self @ g == self @ f`  imply  `g == f` forall g,f 
            https://en.wikipedia.org/wiki/Monomorphism '''
        return np.linalg.matrix_rank(self) == self.dom
    def epic(self):
        ''' is mapping surjective? '''
        return np.linalg.matrix_rank(self) == self.cod
    def product(a,b):
        ''' The product takes in two object (dimensions) a,b and outputs a new dimension d , two projection morphsisms
            and a universal morphism.
            The "product" dimension is the sum of the individual dimensions (funky, huh?)
        '''
        d = a + b # new object
        p1 = FinVect(np.hstack( [np.eye(a)      , np.zeros((a,b))]  ))
        p2 = FinVect(np.hstack( [np.zeros((b,a)), np.eye(b)      ]  ))
        def univ(f,g):
            assert(f.dom == g.dom) # look at diagram. The domains of f and g must match
            e = np.vstack(f,g) 
            # postconditions. True by construction.
            assert( np.allclose(p1 @ e , f )) # triangle condition 1
            assert( np.allclose(p2 @ e , g ) ) # triangle condition 2
            return e
        return d, p1, p2, univ
    def coproduct(a,b):
        ''' The coproduct takes in two object (dimensions) and outputs a new dimension and a universal morphism
        '''
        d = a + b # new object
        p1 = FinVect(np.hstack( [np.eye(a)      , np.zeros((a,b))]  ))
        p2 = FinVect(np.hstack( [np.zeros((b,a)), np.eye(b)      ]  ))
        def univ(f,g):
            assert(f.dom == g.dom) # look at diagram. The domains of f and g must match
            return np.vstack(f,g) 
        return d, p1, p2, univ
        
        return a + b, lambda f,g : np.hstack(f,g) 

    def equalizer(f,g):
        ''' The equalizer is one way of expression an equality problem in the language of Vect.
            It converts an implicit representation of the nullspace into an explicit representation of the 
        '''
        assert( f.dom == g.dom and f.cod == g.cod  ) # Must be parallel morphisms
        return Vect( np.linalg.null_space( np.hstack([f,-g]) )  )
    
    def coequalizer(f,g):
        assert( f.dom == g.dom and f.cod == g.cod  ) # Must be parallel morphisms
        #assert()
        #Vect( scipy.linalg.orth(np.vstack( [f,g] )) )
        pass
    def pullback(f,g):
        assert( f.cod == g.cod  ) # Most have common codomain
        # horizontally stack the two operations. 
        
        null = scipy.linalg.null_space( np.hstack([f,-g]) )
        d = null.shape[1] # dimension object of corner of pullback
        p1 = FinVect(null[:f.dom, :])
        p2 = FinVect(null[f.dom:, :])
        
        def univ(u,v):
            # preconditions
            assert(u.dom == v.dom )
            assert( np.allclose(f @ u , g @ v ) ) # given a new square. This means u,v have to inject into the nullspace
            
            e = null.T @ np.vstack([u,v])  # calculate universal morphism == p1 @ u + p2 @ v
            
            # postcondition
            assert( np.allclose(p1 @ e, u )) # verify triangle 1
            assert( np.allclose(p2 @ e, v ) )  # verify triangle 2
            
            return e    
        
        # postcondition
        assert( np.allclose(  f @ p1,  g @ p2 )  ) # These projections form a commutative square.
        return  d, p1, p2, univ   
    def pushout(f,g):
        assert( f.dom == g.dom ) # Most have common codomain
        pass
    def exponential(self):
        pass
        '''
        The expential object is Hom( V , W -> U ) ~ Hom( V \otimes W , U )
        '''
    def terminal():
        ''' terminal object has unique morphism to it '''
        return 0, lambda x : FinVect(np.ones((0, x)))
    def initial():
        ''' the initial object has a unique morphism from it
        The initial and final object are the same in FinVect'''
        return 0, lambda x :  FinVect(np.ones((x, 0)))

In [28]:

h = FinVect(np.random.randn(4,2))
print(h.monic()) # random matrices going from small d to large d will be injections, monic

f = FinVect(np.random.randn(2,4))
g = FinVect(np.random.randn(2,7))
f.epic()
d, p1, p2, univ = FinVect.pullback(f,g)
d
f @ p1 - g @ p2 

#u = FinVect(np.random.randn(4,2))
#v = FinVect(np.random.randn(7,3))
univ( p1, p2 ).round(5) # identity matrix. Makes sense
univ( p1[ : , :2 ] , p2[ :, :2] ).round(5)
t, u = FinVect.terminal()
print(u(3))
t, u = FinVect.initial()
print(u(4))

True
[]
[]


In [20]:
t, u = FinVect.terminal()
u(3) 

(0, 3)

In [39]:
x, y, z = {i : 2*i for i in range(3)}.values()
y

2

properties

pullback(monic()).monic()

Why are these constructions true? Uh, I'm just convinced they are.


f = FinVect(np.random.randn((2,4)))
g = FinVect(np.random.randn((2,7)))
d, p1, p2, univ = FinVect.pullback(f,g)





In [14]:
Vect(np.eye(3)).par(np.eye(3))

Vect([[1., 0., 0., 0., 0., 0.],
      [0., 1., 0., 0., 0., 0.],
      [0., 0., 1., 0., 0., 0.],
      [0., 0., 0., 1., 0., 0.],
      [0., 0., 0., 0., 1., 0.],
      [0., 0., 0., 0., 0., 1.]])

Vect([[1., 0., 0.],
      [0., 1., 0.],
      [0., 0., 1.]])

Actual functions are very powerful. 
They have both the gift and a curse of having only 1 interface, the ability to apply them

They can syntactically be very small, yet encode in a sense an infinite amount of information


To tabulate a function requires that it be finite

But once you have a tabulation, it is straightforward


In [29]:
from collections import Counter

In [30]:
Counter([1,2,3,4,4,4,4])

Counter({1: 1, 2: 1, 3: 1, 4: 4})

In [40]:
z = {x : x for x in range(10)}
z[0] = 3
all( [count == 1 for count in Counter(z.values()).values()])
type(set(z.keys()))
{z} # we can't have a set of dcitionarys

TypeError: unhashable type: 'dict'

In [42]:
z = {x : x for x in range(10)}
z2 = {x : x for x in range(10)}
z == z2

True

# Computational Category Theory in Python I: Dictionaries for FinSet

Category theory is a mathematical theory with reputation for being very abstract.
Category theory is an algebraic theory of functions. It has the flavor of connecting up little pipes and ports that is reminiscent of dataflow languages or circuits, but with some hearty mathematical underpinnings.

There's a really delightful book by Rydeheard and Burstall called Computational Category Theory. The first time I looked at it, I couldn't make heads or tails of it, going on the double uphill battle of category theory and Standard ML. But looking at it now, it seems extremely straightforward and well presented. It's a cookbook of how to build category theoretic interfaces for software.

I think it is easy to have the miscomprehension that a fancy language like Haskell or Agda is necessary to even begin writing software that encapsulates category theory based ideas, but this is simply not the case. I've been under this misapprehension before.

So I think it is interesting to perform some translation into python, the lingua franca of computing today.

A very simple category is that of finite sets. The objects in the category can be represented by python sets. The morphisms can be represented by python dictionaries. Nothing abstract here. We can rip and tear these things apart any which way we please.

The manipulations are made even more pleasant by the python features of set and dictionary comprehension.


Here is a python class for FinSet.



Next time:

- Numpy - How to compute a pullback in FinVect
- Pandas - There are couple different options here. The Spivak Categorical Databases crew
- Sympy - The category of substition into terms. Also modules
- Z3py - Categorical Logic




For more:
Bartosz Milewski's Category thoery for Programmers


Category theory can get quite high falutin'. 

From a programming perspective what is the point of all this goddamn nonsense?



Good libraries present an interface to the users. One of the measures of the goodness or badness of an interface is how often you are inclined to peek under the hood to get it to do the thing that you need.
I think to some degree that catgeory theorists want to replace underlying mathematical constructions with a uniform categerical interface. 



They abhor piercing the abstraction barrier so much so that they want to replace any underlying mechanisms with a synthetic version.
The formalization and synethetication of math goes all the way back. It has been said that in Euclid's elements, the diagrams are actually just commentary. The arguments in the text should follow given a completely abstract notion of what a point and line actually are. Another exmaple is the square root of two. Most of us have a conception that the square root of 2 is 1.41yada and we can point to it's location on the number line. However, there is a more formal algebraic perspective that the square root of two is some abstract symbol that can be replaced with 2 whenever it appears squared. There is actually a great deal of utility to this persepctive, for example it is the very very start of even framing what it means for a polynomial equation to be solvable in closed form by radicals.


Category theory has been found to be a powerful organizing principle of very disparate fields of mathematics.

I've become used to associating category theory with Haskell and Agda and Coq and Scala and other very complicated powerfully typed languages. But that is not at all a necessity.


I find it very helpful to think about how to meaningfully represent these high falutin' things in python. Python is the lingua franca of programming languages these days. It's pretty easy to learn and get started in all things consider. The library ecosystem is nuts.

We can say that such and such construction exists in this category, but I am very confused unless I can understand how such a thing can be represented and actually calculated. The universal constructions like equalizers and pullbacks require a not impossible, but also not trivial amount of work to actually compute.


FinSet is a very nice category to get started in. 
The first example of a category in a programming context is often the functions of the language itself. This is great because they are very familiar and useful. It's not great because functions are not easily introspectable or analyzable, making some important categorical constructions basically impossible to implement.

Functions from finite domains to finite codomians can be tabulated into the friendly and familiar python dictionary though! We can run those bad boy backwards and forwards, no problem. Slice 'em, dice 'em , fry 'em. In addition we have the love dictionary comprehension syntax, which makes the definitions appear very close to mathematical definitions you'll find on wikipedia.




In [67]:
from collections import Counter
class FinSet():
    def init(self, dom ,cod , f):
        '''  In order to specify a morphism, we need to give a python set that is the domain, a python set 
        that is the codomain, and a dictionary f that encodes a function between these two sets.
        We could by assumption just use f.keys() implicitly as the domain, however the codomain is not inferable from just f.
        In other categories that domain might not be either, so we chose to require both symmettrically.
        '''
        assert( dom == set(f.keys())) # f has value for everything in domain
        assert( all( [y in cod for y in f.value()] )) # f has only values in codomain
        self.cod = cod
        self.dom = dom
        self.f = f
    def __getitem__(self,i):
        # a convenient overloading.
        return self.f[i]
    def compose(f,g):
        ''' Composition is function composition. Dictionary comprehension syntax for the win! '''
        return FinSet( g.dom, f.cod,  { x : f[g[x]]  for x in g.dom })
    def idd(dom):
        '''  The identity morphism on an object dom. A function mapping every x to itself'''
        return FinSet(dom, dom, { x : x for x in dom})
    def __equal__(f,g):
        assert(f.dom == g.dom) # I choose to say the question of equality only makes sense if the arrows are parallel.
        assert(f.cod == g.cod) # ie. they have the same object at head and tail
        return f.f == f.g
    def terminal(dom):
        ''' The terminal object is an object such that for any other object, there is a unique morphism 
        to the terminal object
        This function returns the object itself {()}  and the universal morphism from dom to that object'''
        return {()} , FinSet(dom, {()} ,  {x : () for x in dom} )
    def initial(cod):
        ''' The initial object is an object such that for any other object, there is a unique morphsm 
        from the initial object to that object.
        It is the dual of the terminal object.
        In FinSet, the initial object is the empty set set({}). The mapping is then an empty dictionary dict({})'''
        return set({}) , FinSet(set({}), cod, dict({}))
    def monic(self):
        ''' Returns bool of whether mapping is injective. 
            In other words, maps every incoming element to unique outgoing element.
            In other words, does `self @ g == self @ f`  imply  `g == f` forall g,f 
            https://en.wikipedia.org/wiki/Monomorphism
            Counter class counters occurences'''
        codomain_vals = self.f.values()
        counts = Counter(codomain_vals).values() # https://docs.python.org/3/library/collections.html#collections.Counter
        return all([count == 1 for count in counts]) # no elements map to same element
    def epic(self):
        ''' is mapping surjective? In other words does the image of the map f cover the entire codomain '''
        codomain_vals = self.f.keys() 
        return set(codomain_vals) == self.cod # cover the codomain
    def product(a,b): # takes a sepcific product
        ab = { (x,y) for x in a for y in b } 
        p1 = FinSet( ab, a, { (x,y) : x for (x,y) in ab } )
        p2 = FinSet( ab, b, { (x,y) : y for (x,y) in ab } )
        return ab, p1, p2 , lambda f,g: FinSet( f.dom, ab, { x : (f[x],g[x]) for x in f.dom  } ) # assert f.dom == g.dom, f.cod == a, g.cod == b
    def coproduct(a,b):
        ab = { (0,x) for x in a  }.union({ (1,y) for y in b  })
        i1 = FinSet( a, ab, { x : (0,x)  for x in a } )
        i2 = FinSet( b, ab, { y : (1,y)  for y in b } )
        def fanin(f,g):
            return { (tag,x) : (f[x] if tag == 0 else g[x]) for (tag,x) in ab }
        return ab, i1, i2, fanin
    def equalizer(f,g):
        ''' The equalizer is a construction that allows one to talk about the solution to an equation in a categorical manner
        An equation is f(x) = g(x). It has two mappings f and g that we want to somehow be the same. The solution to this equation
        should be a subset of the shared domain of f and g. Subsets are described from within FinSet by maps that map into the
        subset. 
        '''
        assert(f.dom == g.dom)
        assert(f.cod == g.cod)
        e = { x for x in f.dom if f[x] == g[x] }
        return e, FinSet(e, f.dom, {x : x for x in e})
    def pullback(f,g): # solutions to f(x) = g(y)
        assert(f.cod == g.cod)
        e = {(x,y) for x in f.dom for y in g.dom if f[x] == g[y]} # subset of (f.dom, g.dom) that solves equation
        p1 = FinSet( e, f.dom, { (x,y) : x for (x,y) in e } ) # projection 1
        p2 = FinSet( e, g.dom, { (x,y) : y for (x,y) in e } ) # projection 2
        
        def univ(q1,q2):
            ''' Universal property: Given any other commuting square of f @ q1 == g @ q2, there is a unique morphism
            that injects into e such that certain triangles commute. It's best to look at the diagram'''
            assert(q1.cod == p1.cod) # q1 points to the head of p1
            assert(q2.cod == p2.cod) # q2 points to the head of p2
            assert(q1.dom == q2.dom) # tails of q1 and q2 are the same
            assert(f @ q1 == g @ q2) # commuting square condition
            return FinSet( q1.dom, e ,  { z : ( q1[z] , q2[z] )    for z in q1.dom  }  )
        return e, p1, p2, univ    
    

In [ ]:
   # what about forall and exsists in this category.
   # truth objects 
    def subobject_classifier():
        def univ(mono):
            # for any monomorphism there exists a characteristic function that makes a pullback sqaure with truemorph
            image = set(mono.f.values())
            return FinSet( mono.cod, {True,False}, {  x : (x in image) for x in mono.cod })
            
        return {True, False}, FinSet( {()} , {True, False}, { () : true }) , univ
    
    
    def fan(f,g): # because FinSet is cartesian, product always exists
        assert(f.dom == g.dom)
        return FinSet( f.dom, { (x,y) for x in f.cod for y in g.cod } , { x : (f[x],g[x]) for x in f.dom  } ) 
    def fst(ab):
        return FinSet( ab, a, { (x,y) : x for (x,y) in ab } )
    def snd(ab):
        return FinSet( ab, b, { (x,y) : y for (x,y) in ab } )

    def exponential(f):
        '''
        b^0 = 1
        b^a = b * (b ^ (a - l)) 
        '''
        def all_funs(dom,cod):
            dom = list(dom)
            if len(dom) == 0:
                return set({})
            if len(dom) == 1:
                return {frozenset({(dom[0],cod)}) for y in cod }
            else:
                return { f.union({(dom[0],y)}) for f in all_funs(dom[1:], cod) for y in cod}
        
            
        # we can tabulate finite functions
    def draw():
    
    def # power?
    def # forall
    def # exists

In [3]:
from enum import Enum


# https://ncatlab.org/nlab/show/empty+category
class Zero(Enum):
    pass

# https://ncatlab.org/nlab/show/terminal+category
class One(Enum):
    IdA = 0
    comp_table = {
        IdA : {IdA : IdA}
    }
    
    @property
    def dom(self):
        return self
    @property
    def cod(self):
        return self
    def compose(f,g):
        return comp_table[f][g]

class Two(Enum):
    IdA = 0
    IdB = 1
    F = 3
    dom_table = {
        IdA : IdA,
        IdB : IdB,
        F : IdA
    }
    cod_table = {
        IdA : IdA,
        IdB : IdB,
        F : IdB
    }
    comp_table = {
        IdA : {IdA : IdA},
        IdB : {IdB : IdB,  F : F},
        F :   {IdA : F}
    }
    def dom(self): # represent the objct by it's identity morphism. It's a choice
        return dom_table[self]
        #if self == Two.F:
        #    return Two.IdA
        #else:
        #    return self
    def cod(self):
        cod_table[self]
        
        if self == Two.F:
            return Two.IdB
        else:
            return self
    def idd(x):
        return x
    def compose(f,g):
        assert(f.dom() == g.cod())
        if f == IdA or f == IdB:
            return g
        elif g == IdA or g == IdB:
            return f
    def objects():
        return {IdA, IdB}
            
[morph for morph in Two]


[m for m in Zero]
[m for m in One]
{m for m in Two}

{<Two.F: 3>,
 <Two.IdA: 0>,
 <Two.IdB: 1>,
 <Two.cod_table: {0: 0, 1: 1, 3: 1}>,
 <Two.comp_table: {0: {0: 0}, 1: {1: 1, 3: 3}, 3: {0: 3}}>,
 <Two.dom_table: {0: 0, 1: 1, 3: 0}>}

In [8]:
@dataclass
class Idd():
    obj

class Foo():
    objs = Enum(FooObj, "x y z" )
    A,B,C = range(3)
    morphs = {A,B,C} + {Idd(x) for x in objs}
    
    

NameError: name 'dataclass' is not defined

{
    (F,Id) : F
    (G,Id) : G
    
}

{
    (F,Id,F)
    
}


Set of triples
Dict of tuples
Dict of Dict of
Function


There are also dom and cod tables


@dataclass 
class Idd:
   obj

class OneO(Enum):
class OneM(Enum):

class One():
   A = 1
   B = 2
   C = 3
   A,B,C,D = range(4)
   objs = Enum('OneObj', 'A B C')
   morphs = Enum('OneArr', 'F')
   
   morphs = [A,B,C] + [ Idd(x) for x in objs ]
   
   
   cod_table
   
   assert( set( cod_table.keys()) == set(objs) )
   assert( set( dom_table.keys()) == set(objs) )
   composable_pairs = { (f,g) for f in morphs for g in morphs if f.dom == g.cod  }
   assert( set( comp_table.keys()) == composable_pairs )
   assert(  { compose(compose(f,g),h) = compso como for f in morphs for g in morphs if f.dom == g.cod  } for h in  morphs if f.cod == h.dom   )
   
   
   
   

We need to check that evberything that can be compose is defined
we need to check that everything that is defined can be composed
we need to verify associativity


On way to present a finite catgeory is the give FinSet generators.
{

FinSet()
FinSet()
FinSet

}

In [ ]:
class Z2(Enum): # Th group of Z2. Z mod 2
    Id = 0
    A  = 1
    comp_table = {
        Id : {Id : Id, A : A },
        A  : {Id : A , A : Id}
    }
    def morphisms():
        return {Id, A}
    def objects():
        return {Id}
    def dom(self):
        return Id
    def cod(self):
        return Id
    def compose(f,g):
        if


In [5]:
Two.dom(Two.IdA)

NameError: name 'dom_table' is not defined

In [ ]:
# brute force checking of the axioms of a category



[      for g in Two for f, gf in Two.comp_table[g].items() for h, in Two.comp_table[gf]  ]

In [ ]:
class GraphSchema(Enum):
    IdE = 0
    IdV = 1
    S = 2
    T = 3
    dom_table = {
        IdE : IdE,
        IdV : IdV,
        S : IdE,
        T : IdE
    }
# How is this going to relate to networkx?

In [ ]:
def MonoidPlus():
    def __init__(self, f):
        self.dom = ()
        self.cod = ()
        self.f = f
    def compose(f,g):
        return MonoidPlus(f.f + g.f)
    def idd(unit):
        assert(unit == ())
        return MonoidPlus(0)

class Nat(PlusMonoid):
    pass
    

In [6]:
"asas" + "dasdas"

'asasdasdas'

In [ ]:
class StringMonoid(PlusMonoid): # This is plus monoid
    def __init__(self,):
    
    def compose(f,g):
        return f + g
    def dom(self):
        return ()
    def cod(self):
        return ()

In [ ]:
# paths.
'''
Each networkx graph defines a seperate category PathCat(g)
Morphsism are lists of edges in the graph that meet up.

Paths in spaces are invertible, so they are groupoids.

Homotopy is kind of quotienting by allowable wiggles. Wiggles are higher dimensional paths.

Homotopy is indeed a group given as a set of generators and relations? If we fix the start and end point to be the same.



That's fun

Homology is kind of letting the path become abelian.

'''
class PathCat():
    def __init__(self, g, dom, cod, f):
        self.g = g
        self.dom = dom # domain and cod are vertices in the networkx graph g
        self.f = [e1,e2,e3,e4,e5,e6 ...] # a path as a sequence on the graph. 

'''
Debate: Is it silly to overload IdA as objects?

Could also just have an Id pseudomorphism.
Or Id(A)

@dataclass
class Id():
   obj : Any

class Id():

class FinCat():
   obj = {}
   morph = {}
   cod = {morph : obj}
   dom = {morph : obj}
   

class Presentation()
   obj = {}
   morphgen = {}
   cod
   dom
   relations = {( [f,g,h] , [a,b,c] ) , ... }


    It's a string rewriting problem. If we can get a completion of relation, I'm feelin pretty good.
    
    Oh that's fun. I can use the built in python string functionality.
    
    https://stackoverflow.com/questions/47208509/simple-string-rewriting-system-in-python

The analog of the grobner basis problem should be

we have generators f,g,h, yada (correspond to variable x y z)

we form an ideal of morphisms [fgfgf, fjfjg, fgfg] (closure under left and rgiht compostion of variable set)

Find an equivalanet set of morphsism that generate the same category (which is infinite).




Presentation of a module as both generators and relations
x y z 1 / 


comparisons to computational group theory are interesting

monoids and groups are special cases of categories, so results there apply to catgorical ocnsiderations. At the very least
impossibility results.

Premutation representations are clutch for groups. They are an intrinsic way to talk about a finite gorup
presentations via generators and relations seems in general fishier.


I would think the analog would be a representation fo the category into FinSet
A finite category can always be represented in FinSet via yoneda and pre/post multiplication

def yoneda_embed(fincat, m):
     
    dom = { m2.dom() == m.dom() for m2 in fincat}
    cod = { m2.dom() == m.cod() for m2 in fincat}
    return FinSet(dom, cod, { f : f.comp(m)  for f in dom })
   
    FinSet({ Two.IdA, Two.IdB } , {} , {  for f in dom }  )


So one thing we could do is, you give me an implicitl representation of a category in terms of 
a bunch of Finset morphisms. This is intrinsically bounded in the number of morphisms that exists, however there can be 
a lot
One can't generate new objects, but you can compose and fill out the thing



Huh the Topos of a graph
The truth objects are described in an exercise of burstall

A graph can be described as FinSet^GScema.

What is networkx internal repreentation? It is a bunch of dictionaries


NatTrans
    map[obja] = morphb



Any poset forms a category

In particular a finite poset



differentiable functions form a category
partial functions


objects formulas , morphisms proof.





'''

In [ ]:
class Z3Cat():
    def __init__(dom , cod, given=[]):
        s = Solver()
        s.add(dom)
        s.add(Not(cod))
        s.check()
        # or prove(Implies(dom, cod))
    def idd(dom):
        # prove(dom, dom) this is done in the constructor
        return Z3Cat(dom,dom)
    def compose(f,g):
        prove(f.dom == g.cod) # any equisatisfiable formula are considered equal
        Z3Cat(g.dom, f.cod)
    def fst(a,b)
        #prove( Implies(And(a,b) , a )) # actually this is done in the constructor
        return Z3Cat(And(a,b), b)
    def snd()
    def fan(f,g):
        # prove(f.dom == g.dom) not even sure this assert is strictly necessary
        return Z3Cat( f.dom , And(f.cod,g.cod) , given = [ f , g ]  )
    def initial(a):
        return Z3Cat(False, a)
    def final(a):
        return Z3Cat(a, True)
    def left(a,b):
        return Z3Cat(a,  Or(a,b) )
    def right(a,b):
        return Z3Cat(b,  Or(a,b) )
    def fanin(f,g):
        # assert f.cod == g.cod
        return Z3Cat( Or(f.dom, g.dom), f.cod, given = [f,g] )
    def curry(f): # other side of adjunction
        # extract the terms out
        a = f.dom.args[0]
        b = f.dom.args[1]
        return Z3Cat(  a    ,  Implies(b, f.cod)   ,  given=[f])
    def uncurry(f): # sort of like intros tactic  # on side of adjunction
        # extract the terms out
        # assert f.cod.head == Implies
        a = f.cod.args[0]
        b = f.cod.args[1]
        return Z3Cat(  And(f.dom,a)    ,  b  ,  given=[f])
    def apply(f,x): # sort of modus ponens
        c = f.args[1]
        return Z3Cat( And(f,x), c )
    # propoistional logic so far. How to deal with predicate logic?
    def exists(t, p):
        t2 = FreshConst(t.sort())
        return Z3Cat(p(t) , Exists([t2], p(t2)))
    def exists(t, p):
        t2 = FreshConst(t.sort())
        lambda t: substitute(p, )
        return phole , Z3Cat(p(t) , Exists([t2], p(t2)))
    def foralle(p):
        pfresh = 
        return Z3Cat( p , pfresh  )
    def foralli()
    
    def weakening(): # unused variable
    def dup(a): # Triangle.
        return Z3Cat(a, And(a,a))
        return Z3Cat(Or(a,a), a)
    
    # pullbacks? These do something interesting to free variables?

        

Huh. Z3 ALSO has substitution....
It might be a better choice on multiple fronts.



Endo adjunctions are the easiest to encode. Constructive to fro constructions that take morphisms and output morphism

It is hoped that the a b are inferable, but they may not be. Inferrable is very similar to saying the funcotrs have to be injective on objects, which isn't always the case.
so 
leftadjoint(f, a):
  infer b = f.dom
  return morph(R b, a)

rightadjoint(g,b):
  infer a = g.cod
  return morph(b, L a)


Ok this is fun


f = Function("f") # absratc rpedictae
Z3Cat(leftadjoint(rightadjoint(f)) ,  f)
Z3Cat(f, rightadjoint(leftadjoint(f)) )

Z3 has enough power to possibly in a sense confirm that things are adjoint



Terminal and initial are adjoint to initial and zero category functors. That's a nice example.




In [ ]:
# (a == b, b == c) ==> (a == c)
a, b, c = Int()
Z3Cat( And(a == b, b == c), (a == c))  # goes through of course


Z3Cat.idd(a == b)
Z3Cat.idd(b == c)
Z3Cat.idd( And(a == b, b == c)  ) @ 
#post ocmposition vs pre ocmposiotn - backward forward proof chaining? Yeah. That makes sense.



In [ ]:
class PySet():
    def __init__(dom,cod, f):
        # dom is a input constract. cod is an output contract. They assert false or throw error
        # see Mike Stay javascript
        # f is a python function
        # 
        # we could also use python typing
        # We could also use z3 side conditions. Wouldn't that be fun!
        # it would be very hoare triple like?
    def compose(f,g):
        def res(x): 
            g.dom(x)
            y = g(x)
            g.cod(y) # hmm. cod of g just has to be a subset of dom of f. Don't love that 
            f.dom(y) # we have no way to test for equality of objects really. Literal pointer equality or 
            z = f(y)
            f.cod(z)
            return z

In [83]:
f = (lambda x: x)
f == f # we can do literal equality
(lambda x : x).__code__
int == float

False

In [98]:
{1 : "fred"} == {1 : "fred"}

True

In [96]:
import typing

a = typing.TypeVar("A")
b = typing.TypeVar("B")
dir(a)
dir(typing.Tuple[int,float])
typing.Tuple[int,float].


TypeError: Parameters to generic types must be types. Got 0.

In [ ]:
class FinFunctor():
    # functors whos domain category is finite. These are indexy things.
    # we can discuss diagrams from this perspective. Other stuff. Nat Trans
    # functors whos domain and codomain category are finite?
    def __init__(self, dom, cod, f):
        
class FinNatTrans(): # NatTrans between finfunctors. Can we talk about Limits? In FinSet we can for any ?
    # Propsition: With inital and pullbacks we have all finite limits. This is probably an algorithm
    
    def __init__(self, F, G):
        
class FinLim

class FinColim


In [ ]:
class FinRel():
    
    

In [ ]:
class FinProfunctor():

In [65]:
{1,2,3}.union({3,4,5})

{1, 2, 3, 4, 5}

In [20]:
FinSet({1,2,3}, {4,5,6}, 4)

TypeError: dict expected at most 1 arguments, got 3

In [62]:
frozenset((1,2))

frozenset({1, 2})

In [25]:
type(set({}))

set

The substitution class is very similar to finset in some respects.
We also insist upon a finite set of variables

Now rather than a dictionary lookup, terms are substituted inside expressions.




In [ ]:
class FinSubst():
    '''  Similar in many respects to FinSet '''
    def __init__(dom,cod,f):
        assert( f.keys() == dom )
        assert( all([y.free_symbols in cod for y in f.values()]) )
        self.dom = dom
        self.cod = cod
        self.f = f
    def idd(dom):
        return FinSubst(dom,dom, { x : x for x in dom })
    def compose(f,g):
        #https://docs.sympy.org/latest/modules/core.html#sympy.core.basic.Basic.xreplace
        # not sure xreplace is quite right. Doesn't treat bound variables right. And shared variables between dom and cod could mess things up
        assert(f.dom == g.cod)
        return FinSubst(g.dom , f.cod , { x : f[x].xreplace( g.f ) for x in f.cod  } )
    def equalizer(f,g):
        unify
        

In [ ]:
class SubSet():
    def __init__(dom,cod):
        assert( all( x in cod for x in dom ))
        self.cod = cod
        self.dom = dom
    def compose(f,g):
        assert(f.dom == g.cod)

A inverse image functor.


In [ ]:
class Functor():
    def __init__(self, dom, cod, objmap,fmap):
        self.objmap = objmap #? As Functions?
        self.fmap = fmap
        self.cod = cod
        self.dom = dom
    def idd(C):
        return Functor(C,C, lambda x: x, lambda f : f)
    def compose(f,g):
        assert(f.dom == g.cod)
        return Functor(g.dom, f.cod, lambda x : f.objmap(g.objmap(x)),  lambda x : f.fmap(g.fmap(x)) )
    
        
class ProductCat():
    def __init__(self,f,g):
        self.f = f
        self.g = g
        
Functor( (FinSet , FinSet) ,  FinSet,      )        
        
Functor(FinSet, FinSet,    )    





In [ ]:
class NatTrans():
    def __init__(self,dom,cod, a):
    def idd(F):
        return NatTrans(F,F, lambda o : F.idd(o) )
    def vcompose(a,b):
        assert(a.dom == b.cod)
        return NatTrans(a, b , lambda F : a.a(b.a( F )))
    def hcompose(a,b):
        assert( a.dom.dom ==   )
        

In [ ]:
class Adjunction():
    to = NatTrans( L @ R , Functor.idd() )
    from = NatTrans( Functor.idd()   , R @ L)
    # OR  Hom( L a , b ) ~~ Hom(a, R b)
    

In [28]:
class Product
     a
     b 
     fan

    
g = Product()
g.proj1 = lambda x,y : x

AttributeError: 'dict' object has no attribute 'proj1'

In [ ]:
import typing
class Pyth():
    def __init__(dom, cod, f):
        self.dom = dom
        self.cod = cod
        self.f = f
    def 

In [ ]:
def PreOrd():
    def __init__(self,dom,cod):
        assert(dom <= cod)
        self.dom = dom
        self.cod = cod
    def compose(f,g):
        assert(f.dom == g.cod)
        return PreOrd(g.dom,f.cod)
    def idd(a):
        return PreOrd(a,a)
    

In [ ]:
def MonoidPlus():
    def __init__(self, f):
        self.dom = ()
        self.cod = ()
        self.f = f
    def compose(f,g):
        return MonoidPlus(f.f + g.f)
    def idd(unit):
        assert(unit == ())
        return MonoidPlus(0)

def MonoidTimes():
    def __init__(self, f):
        self.dom = ()
        self.cod = ()
        self.f = f
    def compose(f,g):
        return MonoidTimes(f.f * g.f)
    def idd(unit):
        assert(unit == ())
        return MonoidTimes(1)
    
#def Group(M):
#    def inv(f):
#  maybe 1 / f.f, or - f.f        


# ListMonoid

# 


Orders are a useful example where every morphism is unique (if it exists)



In [ ]:
class Poset():
    def __init__(t, dom, cod ):
        assert(isinstance(cod,t))
        assert(isinstance(dom,t))
        assert(cod <= dom)
        self.cod = cod
        self.com = dom
    def idd(dom):
        Poset(type(dom), dom , dom)
    def compose(f,g)
        assert( f.dom == g.cod )


In [2]:
{1,2,4} <= {1,2,3} # huh. This actually works. So we can do subset as a categroy

# we can also do it for vector subspaces.
# intervals.
# Things that I think about as being the 2-morphisms in relation categroies

False

In [ ]:
class Pand(): # This is a tough one a little bit. What is a database?
    def __init__( dom,cod, f):
        assert( dom in f.columns())
        assert( cod in f.columns())
        # ?  assert(dom + cod in f.columns)
        # assert( intersect(dom,cod) == 0)?
        self.dom = dom
        self.cod = cod
        

In [ ]:
class Op():
    def __init__(self, C, *args):
        self.f = C(*args)
        self.C = C
    def compose(f,g):
        self.C.compose(g,f)
    def idd(dom):
        self.C.idd(dom)
    

In [ ]:
class Graph(): # a category of paths on graph # use networkx. # a category of

[(term, term)] as a category of syntatic relations

In [45]:
hash("hella")

-5139300591851840855

In [9]:
from dataclasses import dataclass

ModuleNotFoundError: No module named 'dataclasses'

In [49]:
@dataclass
class Foo:
    bar : int
    baz : str

    

In [51]:
Foo(4,"seve")

Foo(bar=4, baz='seve')

In [55]:
type(Foo(4,"sev")) == Foo

True

Three valued logic.


Julia implements this. missing

None, True, False

This is a useful construct for missing information


PartialFinSet
 explicitly f[whatever] = None
  or implciilty (which is probably better.

PartialFinSet seems useful for databases with missing entries


unknown has to go to unknown.
Tuple structures. Do they require coherence?



The truth object in PFinSet is {True, False, None}

None is interpreted as unknown.
There are alternate versions of 3 level logic.

four level logic - constradicatory evidence
Perhaps MultiPartialFinsSet? Which are relations. But sort of considering multivalued functions are contradictator








In [ ]:
class PFinSet():
    def __init__():
        # the definition is more symmettircal in many ways
        
    def compose(f,g):
        return PFinSet(g.dom, f.cod, { x : f[g[x]] for x in g.dom if g[x] in f.f })
    
    
# there is a monad resulting from a natural adjunctions between Partial Functions and Functions?
# This is a the maybe monad.


    

In [ ]:
class FinRel():
    def __init__( to , fro): #but better yet, use pandas.
    def __
    
# https://pandas.pydata.org/docs/user_guide/categorical.html
# very amusingly they call this categorical data
    
class FinRel(): # pandas over only enums
    def __init__(dom,  cod):
        dom = { colname : { possible vals }   }

Sympy has a knuth bendix implentation under the hood in the free group module

https://github.com/libsemigroups/libsemigroups_cppyy
https://github.com/sympy/sympy/blob/master/sympy/combinatorics/tests/test_rewriting.py
https://docs.sympy.org/latest/modules/combinatorics/fp_groups.html#constructing-a-presentation-for-a-subgroup
See RFC walters book

In [14]:
class PlusIntMonoid(int):
    def mplus(self,b):
        return self + b
    def mzero():
        return 0

class TimesIntMonoid(int):
    def mplus(self,b):
        return self * b
    def mzero():
        return 1

class ListMonoid(list):
    def mplus(self,b):
        return self + b
    def mzero():
        return []

class UnionMonoid(set):
    def mplus(self,b):
        return self.union(b)
    def mzero():
        return set()

In [18]:
ListMonoid([1,2]).mplus(ListMonoid([1,2]))

[1, 2, 1, 2]

In [8]:
ListMonoid.mzero()

[]

In [10]:
UnionMonoid({1,2}).mplus(UnionMonoid({1,4}))

{1, 2, 4}

In [15]:
UnionMonoid.mzero()

set()

In [17]:
TimesNatMonoid(3).mplus(TimesNatMonoid.mzero())

3

In [19]:
class PlusIntCat(int):
    def compose(self,b):
        return self + b
    def idd():
        return 0
    def dom(self):
        return () # always return (), the only object
    def cod(self):
        return ()

class TimesIntCat(int):
    def compose(self,b):
        return self * b
    def idd():
        return 1
    def dom(self):
        return ()
    def cod(self):
        return ()

class ListCat(int):
    def compose(self,b):
        return self + b
    def idd():
        return []
    def dom(self):
        return ()
    def cod(self):
        return ()

class UnionSetCat(set):
    def compose(self,b):
        return self.union(b)
    def idd(self,b):
        return set()
    def dom(self):
        return ()
    def cod(self):
        return ()
    
PlusIntCat(3).compose(PlusIntCat.idd()) # 3

3

In [ ]:
from sympy.combinatorics.named_groups import SymmetricGroup

G = SymmetricGroup(4)


In [21]:
from sympy.combinatorics.free_groups import free_group, vfree_group, xfree_group
from sympy.combinatorics.fp_groups import FpGroup, CosetTable, coset_enumeration_r
F, a, b = free_group("a, b")
G = FpGroup(F, [a**2, b**3, (a*b)**4])

In [40]:
G.reduce(a*a*a*b*b*b)

AttributeError: 'FreeGroup' object has no attribute 'reduce'

In [66]:
G.__dict__

{'free_group': <free group on the generators (a, b)>,
 'relators': [a**2, b**3, a*b*a*b*a*b*a*b],
 'generators': (a, b),
 'dtype': sympy.combinatorics.fp_groups.FpGroupElement,
 '_coset_table': None,
 '_is_standardized': False,
 '_order': None,
 '_center': None,
 '_rewriting_system': <sympy.combinatorics.rewritingsystem.RewritingSystem at 0x111e91e50>,
 '_perm_isomorphism': None}

In [67]:
vars(G)

{'free_group': <free group on the generators (a, b)>,
 'relators': [a**2, b**3, a*b*a*b*a*b*a*b],
 'generators': (a, b),
 'dtype': sympy.combinatorics.fp_groups.FpGroupElement,
 '_coset_table': None,
 '_is_standardized': False,
 '_order': None,
 '_center': None,
 '_rewriting_system': <sympy.combinatorics.rewritingsystem.RewritingSystem at 0x111e91e50>,
 '_perm_isomorphism': None}

In [34]:
G(a*a*1)

TypeError: only FreeGroup elements of same FreeGroup can be multiplied

In [35]:
help(G)

Help on FpGroup in module sympy.combinatorics.fp_groups object:

class FpGroup(sympy.printing.defaults.DefaultPrinting)
 |  FpGroup(fr_grp, relators)
 |  
 |  The FpGroup would take a FreeGroup and a list/tuple of relators, the
 |  relators would be specified in such a way that each of them be equal to the
 |  identity of the provided free group.
 |  
 |  Method resolution order:
 |      FpGroup
 |      sympy.printing.defaults.DefaultPrinting
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, g)
 |  
 |  __init__(self, fr_grp, relators)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__ = __str__(self)
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  center(self)
 |      Return the list of generators of the center of `self`.
 |  
 |  centralizer(self, other)
 |      Return the list of generators of the centralizer of `other`
 |      (a list of elements of `self`) in `self`.
 |  
 |  coset_enumeration(self, H

In [39]:
G.identity * a * a * a

a**3

In [74]:
from sympy.combinatorics.free_groups import free_group, vfree_group, xfree_group
from sympy.combinatorics.fp_groups import FpGroup, CosetTable, coset_enumeration_r


def fp_group_cat(G, catname):
    # A Category generator that turns a finitely presented group into categorical python class
    Cat = type(catname, (), vars(G))
    def cat_init(self,a):
        self.f = a
    Cat.__init__ = cat_init
    Cat.compose = lambda self,b : G.reduce(self.f * b.f)
    Cat.dom = lambda : ()
    Cat.cod = lambda : ()
    Cat.idd = lambda : Cat(G.identity)
    return Cat

F, a, b = free_group("a, b")
G = FpGroup(F, [a**2, b**3, (a*b)**4])
MyCat = fp_group_cat(G, "MyCat")
MyCat(a*a).compose(MyCat.idd())
MyCat.dom()


()

In [85]:
class IntOrderCat():
    def __init__(self, dom, cod):
        assert(dom <= cod)
        self.cod = cod
        self.dom = dom
        self.f = ()
    def idd(n):
        return IntOrderCat(n,n)
    def compose(f,g):
        assert( f.dom == g.cod )
        return IntOrderCat( g.dom, f.cod )
    def __repr__(self):
        return f"[{self.dom} <= {self.cod}]"
    
# our convention for the order of composition feels counterintuitive here.
IntOrderCat(3,5).compose(IntOrderCat(2,3)) # [2 <= 5]
IntOrderCat.idd(3) # [3 <= 3]

In [94]:
IntOrderCat.idd(3)

[3 <= 3]

In [101]:
class SubSetCat():
    def __init__(self,dom,cod):
        assert( dom.issubset(cod))
        self.cod = cod
        self.dom = dom
    def compose(f,g):
        assert(f.dom == g.cod)
        return SubSetCat(g.dom, f.cod)
    def idd(s):
        return SubSetCat(s,s)
    def __repr__(self):
        return f"[{self.dom} <= {self.cod}]"

SubSetCat(  {1,2,3} , {1,2,3,7} ).compose(SubSetCat( {1,2} , {1,2,3} )) # [{1, 2} <= {1, 2, 3, 7}]

In [102]:
SubSetCat(  {1,2,3} , {1,2,3,7} ).compose(SubSetCat( {1,2} , {1,2,3} ))

[{1, 2} <= {1, 2, 3, 7}]